### Chebyshev

The generic Chebyshev approximation problem is (for $i = 1,...,k$)
\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

We add an L2 term to form the problem:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i \| A^{(i)}x \|_2 + \left|b_i^Tx - c_i\right| \\
  \end{aligned}
\end{equation*}

or equivalently:

\begin{equation*}
  \begin{aligned}
    &\text{minimize} && \max_i [t_i + \left|b_i^Tx - c_i\right| ]\\
    &\text{subject to} && \|A^{(i)}x\|_2 \leq t_i &&&i = 1,...,k
  \end{aligned}
\end{equation*}

In [2]:
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sp.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sp.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])

    return A

np.random.seed(0)
m = 100
n = 200
k = 10
A = [normalized_data_matrix(m,n,1) for i in range(k)]
B = normalized_data_matrix(k,n,1)
c = np.random.rand(k)



In [20]:
# Problem construction
problems = []
opt_vals = []

# Problem 1 (Epigraph form)
x1 = cp.Variable(n)
t = cp.Variable(k)

f = cp.max_entries(t+cp.abs(B*x1-c))
C = []
for i in range(k):
    C.append(cp.pnorm(A[i]*x1, 2) <= t[i])

problems.append(cp.Problem(cp.Minimize(f), C))
opt_vals.append(None)

# Problem 2 (Unconstrained)
x2 = cp.Variable(n)
obj_list = [cp.pnorm(A[i]*x2, 2) + cp.abs(B[i,:]*x2 - c[i]) for i in range(k)]
f2 = cp.max_elemwise(obj_list)

problems.append(cp.Problem(cp.Minimize(f2)))
opt_vals.append(None)

# For debugging individual problems:
if __name__ == "__main__":
    for prob in problems:
        prob.solve(solver = "SCS", eps = 1e-5)
        print("status:", prob.status)
        print("optimal value:", prob.value)
        print("true optimal value:", opt_val)
        for var in prob.variables():
            print(var.value)


('status:', 'optimal')
('optimal value:', 0.3259018468536295)
('true optimal value:', None)
[[ 0.32590123]
 [ 0.26123767]
 [ 0.32590092]
 [ 0.32590281]
 [ 0.32590062]
 [ 0.32590143]
 [ 0.31969286]
 [ 0.3259005 ]
 [ 0.31628078]
 [ 0.32590276]]
[[  5.39108386e-02]
 [ -6.94995743e-03]
 [  2.11077949e-02]
 [ -1.25201169e-02]
 [ -4.00526149e-02]
 [ -1.35700050e-02]
 [ -1.72481835e-03]
 [  3.62936936e-02]
 [  9.37681570e-03]
 [ -5.96198904e-03]
 [  3.63796715e-02]
 [ -3.42700254e-02]
 [  5.74326290e-03]
 [  3.51899209e-02]
 [ -2.15986127e-02]
 [ -4.60421005e-03]
 [  9.76014460e-03]
 [ -2.03798148e-02]
 [ -2.24016993e-02]
 [  1.97817354e-03]
 [  3.30075102e-02]
 [ -6.17123842e-02]
 [ -3.71061020e-03]
 [  4.13163985e-02]
 [ -1.81332730e-02]
 [  1.55175460e-02]
 [ -4.07276781e-03]
 [  6.89917970e-03]
 [  3.57128634e-03]
 [  6.26009022e-02]
 [  2.72243077e-03]
 [  2.99979678e-03]
 [ -3.98413323e-02]
 [  1.84599683e-02]
 [  4.56084601e-02]
 [  1.30690439e-02]
 [  5.88549220e-02]
 [  2.48044265e-0

In [9]:
all(prob.variables()[1].value == prob2.variables()[0].value)

True

<method 'append' of 'list' objects>